In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import math
import pandas as pd

In [17]:
# ethurl = "https://api.etherscan.io/api?module=account&action=balance&address=0xE9df388B01369d972c53ABeb446C5Ae646500222&tag=latest&apikey=FTEB6TINTPNKPMSBJ4IGN9EYR1CAHD2D8A"

# eth_response = requests.get(ethurl, headers=my_headers)
# print(eth_response.json())

{'status': '1', 'message': 'OK', 'result': '2749593091425000'}


In [22]:

def twitter_api(q_string):
    access_token = "AAAAAAAAAAAAAAAAAAAAAJesOgEAAAAAhjjjXkVka1TtQ5lxdzIEJtKk6Tw%3DlkuKd4yJE1x29rdaMdvvvh6BoCxFOL8knOxSWL42QG2nU2auvL"
    my_headers = {'Authorization' : f'Bearer {access_token}'}

    tweet_fields = "&tweet.fields=text&expansions=author_id"
    user_fields = "&user.fields=description,id,location,name,profile_image_url,protected,public_metrics,url,username,verified,withheld"

    response = requests.get(f'https://api.twitter.com/2/tweets/search/recent?query={q_string}{tweet_fields}{user_fields}', headers=my_headers)
    print(f'https://api.twitter.com/2/tweets/search/recent?query={q_string}{tweet_fields}{user_fields}')
    return response.json()


In [3]:
df = pd.read_csv('reddit_scrape/user_addresses_removed_dup.csv')
address_list = df['address'].tolist()
users_list = df['user name'].tolist()
addr_len = len(address_list)

In [7]:
def gen_ugly(i,addr):

    ugly_res = {
        "includes": {
            "users": [
                {
                    "name": f"Kevin J{i}",
                    "url": "",
                    "protected": False,
                    "id": f"484269760_{j}",
                    "description": f"{addr}_{j}",
                    "location": f"Naija{j}",
                    "username": f"jeemmai{j}",
                    "public_metrics": {
                        "followers_count": 40,
                        "following_count": 294,
                        "tweet_count": 619,
                        "listed_count": 1
                    },
                    "profile_image_url": f"{j}https://pbs.twimg.com/profile_images/1319230711245754369/MsxQbjnu_normal.jpg",
                    "verified": False
                }
            ]
        },
        "meta": {
            "newest_id": "1472231060364791812",
            "oldest_id": "1471780673107148802",
            "result_count": 4
        }
    }
    return ugly_res

In [88]:
def makeCalls(ind,df):
    address_list[ind-1] = address_list[ind-1].replace('\n', '').replace('\r','').strip()
    tweet_json = twitter_api(address_list[ind-1])
    #tweet_json = gen_ugly(ind-1,address_list[ind-1])
    print(f"Running call for {ind - 1}")
    if(tweet_json.get("meta").get("result_count") == 0):
        tweet_person = {"description":"", "location":"","name":"","username":"","profile_image_url":""}
    else:
        
        tweet_person = tweet_json.get("includes").get("users")[0]

    tweet_desc.append(tweet_person.get("description"))
    tweet_location.append(tweet_person.get("location"))
    tweet_realname.append(tweet_person.get("name"))
    tweet_username.append(tweet_person.get("username"))
    tweet_avatar.append(tweet_person.get("profile_image_url"))
    print(f'Data -> {address_list[ind-1]}- {tweet_person.get("description")}-{tweet_person.get("location")}-{tweet_person.get("name")}-{tweet_person.get("username")}')
    #data={"user name": users_list[ind], "address": address_list[ind], "description": tweet_desc, "location": tweet_location,"twitter_name": tweet_realname, "twitter_username": tweet_username,"avatar": tweet_avatar}
    # df = df.append(data, ignore_index=True)
    # return df  
    # df = pd.DataFrame(data={"user name": users_list[450:601], "address": address_list[450:601], "description": tweet_desc, "location": tweet_location,"twitter_name": tweet_realname, "twitter_username": tweet_username,"avatar": tweet_avatar})
    # df.to_csv("twitter_data.csv", index=False)


In [89]:
batch_index = 0
max_calls = 450
sleep_time = 15
df = pd.DataFrame()

global_tweet_desc = []
global_tweet_location = []
global_tweet_realname = []
global_tweet_username = []
global_tweet_avatar = []

#math.ceil((addr_len - 1) / max_calls)
for i in range(3,math.ceil((addr_len - 1) / max_calls)):
#for i in range(0,1):
    tweet_desc = []
    tweet_location = []
    tweet_realname = []
    tweet_username = []
    tweet_avatar = []
    print(f"Second loop {i * max_calls} => {addr_len}")
    #for j in range(380 + 1,450):
    for j in range((i * max_calls) + 1,addr_len + 1): # if first +1 is removed, loop will end on 16200 instead of 16374
        makeCalls(j,df)
        print(j)       

        if(j % max_calls == 0):
            break
    
    global_tweet_desc += tweet_desc
    global_tweet_location += tweet_location
    global_tweet_realname += tweet_realname
    global_tweet_username += tweet_username
    global_tweet_avatar += tweet_avatar
    print("Getting to sleep")
    time.sleep(60 * 16)


print("----")
print(len(address_list))
print("BBBBBBB")
print(len(global_tweet_desc))
df = pd.DataFrame(data={"user name": users_list, "address": address_list, "description": global_tweet_desc, "location": global_tweet_location,"twitter_name": global_tweet_realname, "twitter_username": global_tweet_username,"avatar": global_tweet_avatar})
df.to_csv("twitter_data_append.csv", index=False)



Second loop 1350 => 8557
https://api.twitter.com/2/tweets/search/recent?query=0xd02F86c553f4Ea33032dFAA923c180F6c381Cc69&tweet.fields=text&expansions=author_id&user.fields=description,id,location,name,profile_image_url,protected,public_metrics,url,username,verified,withheld
Running call for 1350
Data -> 0xd02F86c553f4Ea33032dFAA923c180F6c381Cc69- ---
1351
https://api.twitter.com/2/tweets/search/recent?query=0x810a5bab8672326ccb155257d7683b9dfa047e0d&tweet.fields=text&expansions=author_id&user.fields=description,id,location,name,profile_image_url,protected,public_metrics,url,username,verified,withheld
Running call for 1351
Data -> 0x810a5bab8672326ccb155257d7683b9dfa047e0d- ---
1352
https://api.twitter.com/2/tweets/search/recent?query=0x6334f03efcadd660343d8f312a06ddac7f7c12cb&tweet.fields=text&expansions=author_id&user.fields=description,id,location,name,profile_image_url,protected,public_metrics,url,username,verified,withheld
Running call for 1352
Data -> 0x6334f03efcadd660343d8f312a0

KeyboardInterrupt: 

In [92]:
df = pd.DataFrame(data={"user name": users_list[1350:8557], "address": address_list[1350:8557], "description": global_tweet_desc, "location": global_tweet_location,"twitter_name": global_tweet_realname, "twitter_username": global_tweet_username,"avatar": global_tweet_avatar})
df.to_csv("twitter_data_append.csv", index=False)